In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import os


all_names = set({})
all_names_list = ['Karlotta','Lena','Dun','Sebi','Robin','Julius','Friedl','Melina','Maxi','Benni','Henning','Mary','Lukas']
n_games_played = np.zeros(len(all_names_list))

all_points = np.zeros((len(all_names_list), len(all_names_list)), dtype=int)
all_wins = np.zeros((len(all_names_list), len(all_names_list)), dtype=int)

date_format = "%d.%m.%Y"  # Define the format matching the input string
date_remove_row = datetime.strptime("13.03.2023", date_format)

if not os.path.isdir("csv_files"):
    os.mkdir("csv_files")

# Load the Excel file
file_path = "Spieltags-Tabelle.xlsx"

# Load the Excel file and check sheet names
excel_data = pd.ExcelFile(file_path, )
print("Sheet Names:", excel_data.sheet_names)

In [52]:
def get_positive_indices(row):
    return row[row > 0].index.tolist()

def get_negative_indices(row):
    return row[row < 0].index.tolist()

In [ ]:
# Process each sheet
for sheet_name in excel_data.sheet_names:
    new_sheet_name = sheet_name.split(" ")[0]
    if len(new_sheet_name.split(".")[2]) == 2:
        new_sheet_name = new_sheet_name[:-2] + "20"+new_sheet_name[-2:]
    game_day = datetime.strptime(new_sheet_name, date_format)
    
    print(f"Processing sheet: {sheet_name}")
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    df.columns = df.columns.str.split(pat=r"[").str[0]  # Keep only the first part
    df.columns = df.columns.str.split().str[0]  # Keep only the first part
        
    # Determine the stopping column based on conditions
    stop_column = len(df.columns)  # Default to all columns
    for i, col_name in enumerate(df.columns):
        if pd.isna(col_name) or col_name == "Bock" or col_name.startswith("Unnamed"):
            stop_column = i
            break

    # Select only the first 4-5 columns or until stopping condition
    df = df.iloc[:, :max(4, stop_column)]
    df = df.dropna(axis=0)
    df = df.astype(int)

    # Remove the rows with number of wins that day
    if game_day > date_remove_row:
        df = df.drop(index=0)

    # How much points were earned this game?
    diff = df.diff()
    # Row 0 is just the first row of the other game
    diff.iloc[0] = df.iloc[0,:]
    diff = diff.astype(int)

    # Add to keep score of names
    for name in df.columns:
        all_names.add(name)
    
    # Save to new file or use further
    output_file = f"csv_files\{new_sheet_name}.csv"

    # Iterate over all games
    for row_idx in range(diff.shape[0]):
        winners_df = get_positive_indices(diff.iloc[row_idx])
        losers_df = get_negative(indices(diff.iloc[row_idx])
        # For the first part ignore Soli
        if len(winners_df) == 2:
            winner1_idx = all_names_list.index(winners_df[0])
            winner2_idx = all_names_list.index(winners_df[1])
            
            all_wins[winner1_idx, winner2_idx] += 1
            all_wins[winner2_idx, winner1_idx] += 1

            n_games_played[winner1_idx] += 1
            n_games_played[winner2_idx] += 1
    
    df.to_csv(output_file, index=False)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
row_sums = all_wins.sum(axis=1)
for i, sum_value in enumerate(row_sums):
    ax.text(all_wins.shape[1]-.5, i, str(sum_value),
            va='center', ha='left', color='black', fontsize=10)

im = plt.imshow(all_wins)
plt.xticks(ticks=np.arange(len(all_names_list)),
           labels=all_names_list, 
           rotation=90)
plt.yticks(ticks=np.arange(len(all_names_list)),
           labels=all_names_list
          )
plt.colorbar(im)
plt.show()